In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
## Read file by city
csvfile = './Resources/colonias.csv'
colonias_df = pd.read_csv(csvfile, encoding='utf-8-sig')
colonias_df.head()

,id,nombre,municipio,asentamiento,codigo_postal,latitud,longitud
0,10011,Zona Centro,1001,Colonia,20000,21.881191,-102.296143
1,10012,Colinas del Rio,1001,Fraccionamiento,20010,21.894022,-102.313606
2,10013,Olivares Santana,1001,Colonia,20010,21.894022,-102.313606
3,10014,Las Brisas,1001,Fraccionamiento,20010,21.894022,-102.313606
4,10015,Ramon Romo Franco,1001,Fraccionamiento,20010,21.894022,-102.313606


In [3]:
# remove spaces from columns names and "'" from data
colonias_df.columns = colonias_df.columns.str.replace(' ', '')


In [4]:
len(colonias_df)

143852

In [5]:
# look at columns types
colonias_df.dtypes

id                 int64
nombre            object
municipio          int64
asentamiento      object
codigo_postal      int64
latitud          float64
longitud         float64
dtype: object

In [6]:
municipios_df = pd.read_csv('./Resources/municipios.csv')
municipios_df.columns = municipios_df.columns.str.replace(' ', '')
municipios_df.head()

,id,nombre,estado,cp_min,cp_max,huso_horario
0,1001,Aguascalientes,1,20000,20396,Tiempo del Centro
1,1002,Asientos,1,20700,20799,Tiempo del Centro
2,1003,Calvillo,1,20800,20890,Tiempo del Centro
3,1004,Cosío,1,20460,20478,Tiempo del Centro
4,1005,Jesús María,1,20900,20997,Tiempo del Centro


In [7]:
len(municipios_df)

2465

In [8]:
municipios_df.dtypes

id               int64
nombre          object
estado           int64
cp_min           int64
cp_max           int64
huso_horario    object
dtype: object

In [9]:
colonias_df['nombreMun'] = colonias_df['municipio'].map(municipios_df.set_index('id')['nombre'])
colonias_df.head()

,id,nombre,municipio,asentamiento,codigo_postal,latitud,longitud,nombreMun
0,10011,Zona Centro,1001,Colonia,20000,21.881191,-102.296143,Aguascalientes
1,10012,Colinas del Rio,1001,Fraccionamiento,20010,21.894022,-102.313606,Aguascalientes
2,10013,Olivares Santana,1001,Colonia,20010,21.894022,-102.313606,Aguascalientes
3,10014,Las Brisas,1001,Fraccionamiento,20010,21.894022,-102.313606,Aguascalientes
4,10015,Ramon Romo Franco,1001,Fraccionamiento,20010,21.894022,-102.313606,Aguascalientes


In [10]:
len(colonias_df)

143852

In [11]:
estados_df = pd.read_csv('./Resources/estados.csv')
estados_df.columns = estados_df.columns.str.replace(' ', '')
estados_df.head()

,clave,nombre,abreviacion,cp_min,cp_max
0,1,Aguascalientes,AGU,20000,20999
1,2,Baja California,BCN,21000,22999
2,3,Baja California Sur,BCS,23000,23999
3,4,Campeche,CAM,24000,24999
4,5,Coahuila de Zaragoza,COA,25000,27999


In [12]:
estados_df.dtypes

clave           int64
nombre         object
abreviacion    object
cp_min          int64
cp_max          int64
dtype: object

In [13]:
#colonias_df['entidad'] = colonias_df['codigo_postal'].map(lambda x: estados_df.set_index('id')['nombre'] if x >= estados_df.set_index('clave')['cp_min'] and x <= estados_df.set_index('clave')['cp_min'])

In [14]:
estadoColumn = []
for index, row in colonias_df.iterrows():
    for jndex, row2 in estados_df.iterrows():
        if row["codigo_postal"] >= row2["cp_min"] and row["codigo_postal"] <= row2["cp_max"]:
            estadoColumn.append(row2["nombre"])

In [15]:
len(estadoColumn)

143852

In [16]:
colonias_df.rename(columns={'id': 'id_locations'}, inplace=True)

In [18]:
colonias_df['estado'] = estadoColumn
colonias_df['estado'] = colonias_df['estado'].str.strip()
colonias_df.head()

,id_locations,nombre,municipio,asentamiento,codigo_postal,latitud,longitud,nombreMun,estado
0,10011,Zona Centro,1001,Colonia,20000,21.881191,-102.296143,Aguascalientes,Aguascalientes
1,10012,Colinas del Rio,1001,Fraccionamiento,20010,21.894022,-102.313606,Aguascalientes,Aguascalientes
2,10013,Olivares Santana,1001,Colonia,20010,21.894022,-102.313606,Aguascalientes,Aguascalientes
3,10014,Las Brisas,1001,Fraccionamiento,20010,21.894022,-102.313606,Aguascalientes,Aguascalientes
4,10015,Ramon Romo Franco,1001,Fraccionamiento,20010,21.894022,-102.313606,Aguascalientes,Aguascalientes


In [19]:
file = "./Resources/locationsMexico.csv"
colonias_df.to_csv(file, index=False, encoding='utf-8-sig')

In [20]:
colonias_df = pd.read_csv('./Resources/locationsMexico.csv', encoding='utf-8-sig')

### Interaction with SQLite

In [21]:
typesCols = []

In [22]:
# Create type for each columns to make DB Table 'Inmuebles'
for i in colonias_df.dtypes.values:
    if i == 'int64':
        typesCols.append('integer')
    elif i == 'float64':
        typesCols.append('float')
    else:
        typesCols.append('string')

In [23]:
# Create string for table columns and types
colsDBlist = [str(x[0]) + " " + str(x[1]) for x in zip(colonias_df.columns.values, typesCols)]
colsDB = ",".join(colsDBlist)

In [24]:
# connect to database
conn = sqlite3.connect('./Resources/finalData.db')
c = conn.cursor()

In [25]:
# create table 'Locations' in databse

c.execute('''SELECT count(name) FROM sqlite_master WHERE type='table' AND name='Locations' ''')

#if the count is 1, then table exists
if c.fetchone()[0]==1: 
    print('Table exists.')
    c.execute("DROP TABLE Locations")
    print("Table dropped... ")
    conn.commit()
    columns = colsDB
    queryCreate = f"CREATE TABLE Locations ({columns})"
    c.execute(queryCreate)
    print('New table created.')
else:
    print('Table does not exist.')
    columns = colsDB
    queryCreate = f"CREATE TABLE Locations ({columns})"
    c.execute(queryCreate)
    print('Table created.')

Table exists.
Table dropped... 
New table created.


In [26]:
# write the data to a sqlite table
colonias_df.to_sql("Locations", conn, if_exists='append', index = False)
conn.close()